In [1]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.applications.resnet50 import ResNet50

In [2]:
def readDatasetDF(datasetType, rootDir='data/'):
    """
    dataset Type: "train", "val", "test" 
    Return:
        dataframe with image name/label
    """
    df = pd.read_csv('{}/{}-calibrated-shuffled.txt'.format(rootDir, datasetType), header=None, delimiter = " ")
    df[0] = rootDir+"/"+df[0]
    return df

def createNumpyData(df, RESIZE=64):
    """
    Return:
        img_lst.shape = (n, RESIZE, RESIZE, num_channel)
        label_lst.shape = (n, 1)
    """
    img_lst = np.array([])
    label_lst = []
    RESIZE = 64

    for i, row in df.iterrows():
        img = cv2.imread(row[0]) / 255
        img = cv2.resize(img, (RESIZE, RESIZE))
        label = row[1]

        # append data to img_lst
        if len(img_lst)==0:
            img_lst = np.expand_dims(img, axis=0)
        else:
            img_lst = np.concatenate((img_lst, np.expand_dims(img, axis=0)), axis=0)
        # append data to label_lst
        label_lst.append(label)
        if i%100==0:
            print(i, end=', ')
    # change shape of label_lst from (n,) to (n,1)
    label_lst = np.expand_dims(np.array(label_lst), axis=-1)
    
    img_lst = img_lst.astype('float32')
    label_lst = label_lst.astype('uint8')
    
    return img_lst, label_lst

In [3]:
df_train = readDatasetDF('train')
x_train, y_train = createNumpyData(df_train)

0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 

In [10]:
x_train.shape

(3746, 64, 64, 3)

In [9]:
y_train.shape

(3746, 1)

In [38]:
W, H = 64, 64
resnet = ResNet50(include_top=False, weights='imagenet')

for layer in resnet.layers:
    layer.trainable = False

num_classes = 25
    
x = resnet.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x) 
predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.Model(resnet.input, predictions)

In [39]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [40]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
118/118 [==============================] - 10s 74ms/step - loss: 1.8984 - accuracy: 0.5624
Epoch 2/10
118/118 [==============================] - 9s 74ms/step - loss: 1.4801 - accuracy: 0.6220
Epoch 3/10
 93/118 [======================>.......] - ETA: 1s - loss: 1.2826 - accuracy: 0.6830

KeyboardInterrupt: 